In [1]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp
pip install jupyterthemes

  Using cached pip-20.2.4-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=a30fed69898a03a060dda39ee5dbb6043aa75aa2e27c34eadfb9fb7b53005acd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/88/2a/d4/f2e9023989d4d4b3574f268657cb6cd23994665a038803f547
Successfully built smart-open


In [2]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [3]:
import time 
import re
import os
import trp
import random 
import boto3
import minecart
import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session
from io import BytesIO

%matplotlib inline

In [4]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [9]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2)
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # itterate through each row within the provided table
    for row in range(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in range(M)]
        
    return pd.DataFrame(arr)

In [55]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        # itterate through page tables
        for table in page.tables: 
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty (column 0 = line items)
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
            
            # make sure dataframe is not empty  
            if df.values.size > 0:
                colIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity
                assetCheck = df[colIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity
                debtCheck1 = df[colIndex].str.contains('Liabilities$|^Liabilities', 
                                                      regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[colIndex].str.contains('Liability$|^Liability', 
                                                      regex=True, flags=re.IGNORECASE)

                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty
                check2 = debtCheck1[debtCheck1 == True].empty
                check3 = debtCheck2[debtCheck2 == True].empty

                # if figure matched exactly, we assume this is balance sheet 
                if not check1:
                    catDF.append(df)

                    if check2 == False or check3 == False:  
                        return pd.concat(catDF)
                

## Extract Balance Sheet information

In [11]:
subsetFolder = np.array(session.list_s3_files(bucket, 'Input/SubSets/'))[1:]

In [12]:
subsetFolder

array(['Input/SubSets/1000147-02_subset.pdf',
       'Input/SubSets/1000147-03_subset.pdf',
       'Input/SubSets/1000147-04_subset.pdf', ...,
       'Input/SubSets/99947-17_subset.pdf',
       'Input/SubSets/99947-18_subset.pdf',
       'Input/SubSets/99947-19_subset.pdf'], dtype='<U35')

In [41]:
print('Total files needed to be converted {}'.format(subsetFolder.size))

Total files needed to be converted 78504


In [57]:
# script to perform OCR (using Textract) for X-17A-5 subsets
outFolder = 'Output/BalanceSheet/'
csvDirectory = np.array(session.list_s3_files(bucket, outFolder))
errorLog = []

# iterate through X-17A-5 subsets stored in s3
for i, key in enumerate(subsetFolder[:10000]):
    # baseFile name (CIK)-{Year}
    baseFile = key.split('/')[-1].split('_')[0]
    fileName = baseFile+'.csv'
    print('\nFile {}'.format(i))
    # if file is not found in directory 
    if outFolder+fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", key)
        
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            # checks for type of return 
            if type(tempDF) == pd.DataFrame:
                # writing data frame to .csv file
                tempDF.to_csv(fileName)

                # the folder used to export to
                xFolder = 'Output/BalanceSheet/'+fileName

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=xFolder, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                print('\tSaved {} file to s3 bucket'.format(baseFile+'.csv'))
            else:
                print('{} no Balance Sheet found'.format(baseFile))
                errorLog.append(baseFile)
        else:
            print('{} could not be parsed'.format(baseFile))
            errorLog.append(baseFile)
    else:
        print('{} has been downloaded'.format(fileName))

print('==========================\nOCR is completed')


File 0
1000147-02.csv has been downloaded

File 1
1000147-03.csv has been downloaded

File 2
1000147-04.csv has been downloaded

File 3
1000147-05.csv has been downloaded

File 4
1000147-06.csv has been downloaded

File 5
1000147-07.csv has been downloaded

File 6
1000147-08.csv has been downloaded

File 7
1000147-09.csv has been downloaded

File 8
1000147-10.csv has been downloaded

File 9
1000147-11.csv has been downloaded

File 10
1000148-02.csv has been downloaded

File 11
1000148-03.csv has been downloaded

File 12
1000148-04.csv has been downloaded

File 13
1000148-05.csv has been downloaded

File 14
1000148-06.csv has been downloaded

File 15
1000148-07.csv has been downloaded

File 16
1000148-08.csv has been downloaded

File 17
1000148-09.csv has been downloaded

File 18
1000148-10.csv has been downloaded

File 19
1000148-11.csv has been downloaded

File 20
1000148-12.csv has been downloaded

File 21
1000148-13.csv has been downloaded

File 22
1000148-14.csv has been downloade

Started job with id: 8ef35b85a4f8f266600fec974bdcacefe8320b0ca9df30c120f2f4e2aebc8f48
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
1001015-13 no Balance Sheet found

File 151
1001015-14.csv has been downloaded

File 152
1001015-15.csv has been downloaded

File 153
1001015-16.csv has been downloaded

File 154
1001015-17.csv has been downloaded

File 155
1001017-02.csv has been downloaded

File 156
1001017-03.csv has been downloaded

File 157
1001017-04.csv has been downloaded

File 158
1001017-05.csv has been downloaded

File 159
1001017-06.csv has been downloaded

File 160
1001017-07.csv has been downloaded

File 161
Started job with id: 174372267d074cf67060278695a5c5967a1bc8c7bcec81e8921

Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
1001379-16 no Balance Sheet found

File 306
1001379-17.csv has been downloaded

File 307
1001379-18.csv has been downloaded

File 308
1001379-19.csv has been downloaded

File 309
1001382-02.csv has been downloaded

File 310
1001382-03.csv has been downloaded

File 311
1001382-04.csv has been downloaded

File 312
1001382-05.csv has been downloaded

File 313
1001382-06.csv has been downloaded

File 314
1001382-07.csv has been downloaded

File 315
1001382-08.csv has been downloaded

File 316
1001382-09.csv has been downloaded

File 317
1001382-10.csv has been downloaded

File 318
1001382-11.csv has been downloaded

File 319
1001382-12.csv has been downloaded

File 320
1001382-13.csv has been downloaded

File 321
1001382-15.csv has been downloaded

File 322
1001382-16.csv has been downloaded

File 323
1001382-17.csv has been downloaded

File 324
1001382-18.csv has been downloaded

File 325
1001382-19.csv has been

Started job with id: 2e30ba971e988e2fc98763b6611706971a84841d88666244b58443d3f0eea78a
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    1       2            3
0                                              Assets                     
1                           Cash and cash equivalents             $ 55,473
2                                 Securities borrowed               72,285
3   Receivable from brokers, dealers, and clearing...               70,482
4   Receivable from clients - net of allowance for...  $9,231    3,492,836
5                          Receivable from affiliates               11,150
6                                    Securities owned                5,732
7                                        Other assets               25,009
8                          Deferred income taxes, net           

Started job with id: ed698f21e9790db6486c50d7befd6ae2eb9c7ea5fa02bc9a84fb06dc098ec988
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                        0            1
0                                  Assets             
1                                                     
2                                    Cash  $ 1,011,891
3     Securities owned at market (note 2)      612,498
4                             Receivables             
5                         Clearing broker       29,317
6        Advances to employees/affiliates       50,000
7                    Investments (note 5)      549,553
8                   Deferred tax (note 4)       26,978
9                            Total assets  $ 2,280,237
10        Liabilities and Member's Equity             
11                            Liabilities             
12                      

...

Started job with id: cc38fca61bc27240037895b216a760f6c1d5eeb99fb0a1c2a279f80d7e728e00
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0          1
0                 Receivable from brokers and dealers    329,474
1        

	Saved 1002807-14.csv file to s3 bucket

File 688
Started job with id: 862373e412b22aa17a00424361241a84d524b881c93ae0875fd1c16204014373
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
                                                    0         1
0                                                          2014
1                                              Assets          
2                           Cash and cash equivalents   $ 2,721
3                          Deposit with clearing firm    25,000
4                      Receivable from broker/dealers    49,836
5                                        Other assets       937
6

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                                    0            1
0                                                Cash  $ 1,153,613
1                            Due from clearing broker       88,237
2                        Deposit with clearing broker      100,000
3                                 Accounts Receivable       46,372
4   Advances to brokers - net of loss reserve of $...    1,224,715
5                         SDN Collateral Market Value      250,000
6                                  Right of use Asset      821,842
7                              Due from related party       55,400
8                                    Prepaid expenses       57,680
9   Furniture and equipment, net of accumulated de...       31,131
10                                       Other Assets       16,306
11                                       Total a

	Saved 1002808-06.csv file to s3 bucket

File 698
Started job with id: f1b56fca594a2504d009eea5319fed948092c133f1a9d7f4068f911af95393d8
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS


KeyboardInterrupt: 

In [44]:
np.array(errorLog)

array(['1000148-16', '1000151-03', '1000151-04', '1000151-12',
       '1000316-16', '1001015-11', '1001015-13', '1001017-08',
       '1001023-02', '1001023-03', '1001023-07', '1001360-09',
       '1001361-09', '1001373-04', '1001379-16', '1001710-05',
       '1001723-09', '1001723-10'], dtype='<U10')

# Helper Functions

In [29]:
def cleanNumeric(value:str) -> float:
    """
    This function converts a string to a numeric quantity, handles weird string format
    :param: value, string value with hidden numeric quanity  
    :return: floating point values
    
    Complexity -> O(n)
    
    e.g.
        In[0]: $ 19,225     ->   Out[0]: 19255
        In[0]: $ 19,225.76  ->   Out[0]: 19255.76
    """
    # perform regex operation scanning for only numeric quantities/identifiers
    cleanValue = re.sub("[^0-9|.|-]", "", value)
    if cleanValue in ['', '.', '-']:
        return 0.0
    else:
        return float(cleanValue)

In [39]:
cleanNumeric('-$129.25') + 20

-109.25